# DTSA5511 - GANs Monet Painting 

**Author** - Korkrid Akepanidtaworn, University of Colorado Boulder, Masters in Data Science
**Date** - August, 20,2024

## Project Goal

- Computer vision has advanced tremendously in recent years and GANs are now capable of mimicking objects in a very convincing way. But creating museum-worthy masterpieces is thought of to be, well, more art than science. So can (data) science, in the form of GANs, trick classifiers into believing you’ve created a true Monet? That’s the challenge you’ll take on!
- My goal is to build a GAN that generates 7,000 to 10,000 Monet-style images. 
- A GAN consists of at least two neural networks: a generator model and a discriminator model. The generator is a neural network that creates the images. For our competition, you should generate images in the style of Monet. This generator is trained using a discriminator.
- The two models will work against each other, with the generator trying to trick the discriminator, and the discriminator trying to accurately classify the real vs. generated images.
- [I’m Something of a Painter Myself | Kaggle](https://www.kaggle.com/competitions/gan-getting-started)

## Dataset Description

**Dataset Structure**

The dataset is organized into four directories:
* `monet_tfrec`: Contains Monet paintings in TFRecord format (256x256).
* `monet_jpg`: Contains Monet paintings in JPEG format (256x256).
* `photo_tfrec`: Contains photos in TFRecord format (256x256).
* `photo_jpg`: Contains photos in JPEG format (256x256).

**Data Equivalence**

* `monet_tfrec` and `monet_jpg` directories contain identical Monet painting images.
* `photo_tfrec` and `photo_jpg` directories contain identical photo images.

**Recommended Data Format**

While JPEG images are provided, we strongly recommend using TFRecords as a starting point. Working with TFRecords can be a valuable learning experience for a new data format.

**Dataset Purpose**

* **Monet directories:** Utilize the Monet paintings for training your model.
* **Photo directories:** Apply a Monet-style transformation to these photos and submit the generated JPEG images in a zip file. A maximum of 10,000 images is allowed.

**Submission Guidelines**

* The submitted images can be entirely generated Monet-style art without relying solely on transformed photos.
* Explore other GAN architectures like DCGAN for creating Monet-style art from scratch.
* Consider using the CycleGAN dataset to experiment with different artistic styles.

**File Details**

* `monet_jpg`: 300 Monet paintings (256x256 JPEG)
* `monet_tfrec`: 300 Monet paintings (256x256 TFRecord)
* `photo_jpg`: 7028 photos (256x256 JPEG)
* `photo_tfrec`: 7028 photos (256x256 TFRecord)

In [ ]:
# Check Python version
!python --version

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Standard libraries
import random  # For generating random numbers and shuffling
import shutil  # For file operations like copying and moving

# Numerical and data manipulation libraries
import numpy as np  # For numerical operations and arrays
import pandas as pd  # For data manipulation and analysis

# Image handling and visualization libraries
import matplotlib.image as mpimg  # For reading image files
import matplotlib.pyplot as plt  # For creating plots and visualizations
from PIL import Image  # For opening and manipulating images

# OS and file path utilities
import os
from os import listdir  # For listing files in a directory
from os.path import isfile, join  # For path manipulations and checking file existence

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# List all the files in the specified directory
os.listdir('../input/gan-getting-started/')

In [ ]:
# Define directories containing images
monet_dir = '../input/gan-getting-started/monet_jpg/'
photo_dir = '../input/gan-getting-started/photo_jpg/'

# Retrieve filenames for each directory
monet_filenames = [join(monet_dir, f) for f in listdir(monet_dir) if isfile(join(monet_dir, f))]
photo_filenames = [join(photo_dir, f) for f in listdir(photo_dir) if isfile(join(photo_dir, f))]

# Load images and convert them to NumPy arrays
monet_files = [np.array(Image.open(f)) for f in monet_filenames]
photo_files = [np.array(Image.open(f)) for f in photo_filenames]

# Print metrics about the loaded images
print(f"Number of Monet files: {len(monet_files)}")
print(f"Number of Photo files: {len(photo_files)}")
print(f"Example image shape: {monet_files[1].shape}")

## EDA Procedure

Show the sample image from the 13th.

In [ ]:
# Display examples of images
plt.figure(figsize=(10, 5))

# Display a sample photo
plt.subplot(1, 2, 1)
plt.title('Photo')
plt.imshow(mpimg.imread(photo_filenames[13]))
plt.axis('off')  # Hide axes for cleaner visualization

# Display a sample Monet painting
plt.subplot(1, 2, 2)
plt.title('Monet')
plt.imshow(mpimg.imread(monet_filenames[13]))
plt.axis('off')  # Hide axes for cleaner visualization

plt.show()

Show the sample image from the 15th.

In [ ]:
# Display examples of images
plt.figure(figsize=(10, 5))

# Display a sample photo
plt.subplot(1, 2, 1)
plt.title('Photo')
plt.imshow(mpimg.imread(photo_filenames[15]))
plt.axis('off')  # Hide axes for cleaner visualization

# Display a sample Monet painting
plt.subplot(1, 2, 2)
plt.title('Monet')
plt.imshow(mpimg.imread(monet_filenames[15]))
plt.axis('off')  # Hide axes for cleaner visualization

plt.show()

Let's write a loop to preview data (as necessary.)

In [ ]:
for i in [1,2,3]:
    
    # Display examples of images
    plt.figure(figsize=(10, 5))

    # Display a sample photo
    plt.subplot(1, 2, 1)
    plt.title('Photo')
    plt.imshow(mpimg.imread(photo_filenames[i]))
    plt.axis('off')  # Hide axes for cleaner visualization

    # Display a sample Monet painting
    plt.subplot(1, 2, 2)
    plt.title('Monet')
    plt.imshow(mpimg.imread(monet_filenames[i]))
    plt.axis('off')  # Hide axes for cleaner visualization

    plt.show()

## Helper Functions

In [ ]:
class TorchDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform
        self.image_paths = [os.path.join(data_path, file) for file in os.listdir(data_path) if file.endswith('.jpg')]
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
MonetDataset = TorchDataset(data_path='/kaggle/input/gan-getting-started/monet_jpg', transform=transforms)
PhotoDataset = TorchDataset(data_path='/kaggle/input/gan-getting-started/photo_jpg', transform=transforms)

MonetLoader = DataLoader(MonetDataset, batch_size=16, shuffle=True, num_workers=2)
PhotoLoader = DataLoader(PhotoDataset, batch_size=16, shuffle=True, num_workers=2)

In [ ]:
print(MonetDataset)
print(PhotoDataset)

In [ ]:
print(MonetLoader)
print(PhotoLoader)

In [ ]:
# Inspect the monet tensor
print(next(iter(MonetDataset)))

In [ ]:
# Inspect the photo tensor
print(next(iter(PhotoDataset)))

In [ ]:
monets = next(iter(MonetLoader))
photos = next(iter(PhotoLoader))
print(monets.shape, photos.shape)

# tansform data to be readable by plt.imshow
monets_imshow = []
for monet in monets:
    monet = monet.permute(1, 2, 0) 
    monet = monet.numpy() 
    monet = monet * 0.5 + 0.5 
    monets_imshow.append(monet)

photos_imshow = []    
for photo in photos:
    photo = photo.permute(1, 2, 0) 
    photo = photo.numpy()
    photo = photo * 0.5 + 0.5
    photos_imshow.append(photo)
    
print(monets_imshow[3].shape) 
print(monets_imshow[3].dtype) 

In [ ]:
print('Monet')
plt.figure(figsize=(15, 5)) 
for i in range(4):
    plt.subplot(1, 4, i+1)  
    plt.imshow(monets_imshow[i])
plt.show()

print('Photos')
plt.figure(figsize=(15, 5)) 
for i in range(4):
    plt.subplot(1, 4, i+1)  
    plt.imshow(photos_imshow[i])
plt.show()

## Model Building and Training

### Build the Generator

![](https://hardikbansal.github.io/CycleGANBlog/images/model.jpg)

High level structure of Generator can be viewed in the following image.

![](https://hardikbansal.github.io/CycleGANBlog/images/Generator.jpg)

In [ ]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        
        if down:
            conv_layer = nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
        else:
            conv_layer = nn.ConvTranspose2d(in_channels, out_channels, **kwargs)
        
        if use_act:
            act_layer = nn.ReLU(inplace=True)
        else:
            act_layer = nn.Identity()
        
        self.cnn = nn.Sequential(
            conv_layer,
            nn.InstanceNorm2d(out_channels),
            act_layer,
        )

    def forward(self, x):
        return self.cnn(x)


class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            CNNBlock(channels, channels, kernel_size=3, padding=1),
            CNNBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)

In [ ]:
class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                img_channels,
                num_features,
                kernel_size=7,
                stride=1,
                padding=3,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                CNNBlock(
                    num_features, 
                    num_features * 2, 
                    kernel_size=3, 
                    stride=2, 
                    padding=1
                ),
                CNNBlock(
                    num_features * 2,
                    num_features * 4,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                ),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features * 4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                CNNBlock(
                    num_features * 4,
                    num_features * 2,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                CNNBlock(
                    num_features * 2,
                    num_features * 1,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
            ]
        )

        self.last = nn.Conv2d(
            num_features * 1,
            img_channels,
            kernel_size=7,
            stride=1,
            padding=3,
            padding_mode="reflect",
        )

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))

In [ ]:
img_channels = 3
img_size = 256

test_input = torch.randn((2, img_channels, img_size, img_size))

model_gen_test = Generator(img_channels, 9)

print(model_gen_test(test_input).shape)

### Build the Discriminator

In [ ]:
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.cnn = nn.Sequential(
            # Convolution layer
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size=4,
                stride=stride,
                padding=1,
                bias=True,
                padding_mode="reflect",
            ), 
            nn.InstanceNorm2d(out_channels), 
            nn.LeakyReLU(0.2, inplace=True), 
        )
    
    def forward(self, x):
        return self.cnn(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                Block(in_channels, feature, stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature
        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))

In [ ]:
model_dis_test = Discriminator()

test_input = torch.randn((2, 3, 256, 256))

print(model_dis_test(test_input).shape)

### Train the Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
from torch.utils.data import DataLoader, random_split

# Calculate sizes for train and validation splits
val_size_monet = int(0.2 * len(MonetDataset))
train_size_monet = len(MonetDataset) - val_size_monet

val_size_photo = int(0.2 * len(PhotoDataset))
train_size_photo = len(PhotoDataset) - val_size_photo

# Split datasets into training and validation sets
train_MonetDataset, val_MonetDataset = random_split(MonetDataset, [train_size_monet, val_size_monet])
train_PhotoDataset, val_PhotoDataset = random_split(PhotoDataset, [train_size_photo, val_size_photo])

# Create DataLoaders for the training and validation sets
train_MonetLoader = DataLoader(train_MonetDataset, batch_size=16, shuffle=True, num_workers=2)
val_MonetLoader = DataLoader(val_MonetDataset, batch_size=16, shuffle=False, num_workers=2)

train_PhotoLoader = DataLoader(train_PhotoDataset, batch_size=16, shuffle=True, num_workers=2)
val_PhotoLoader = DataLoader(val_PhotoDataset, batch_size=16, shuffle=False, num_workers=2)

In [ ]:
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
G_Photo2Monet = Generator(img_channels=3).to(device)
G_Monet2Photo = Generator(img_channels=3).to(device)
D_Photo = Discriminator(in_channels=3).to(device)
D_Monet = Discriminator(in_channels=3).to(device)

optimizer_G_Photo2Monet = optim.Adam(G_Photo2Monet.parameters(), lr=1e-4, betas=(0.5, 0.5))
optimizer_G_Monet2Photo = optim.Adam(G_Monet2Photo.parameters(), lr=1e-4, betas=(0.5, 0.5))
optimizer_D_Photo = optim.Adam(D_Photo.parameters(), lr=1e-4, betas=(0.5, 0.5))
optimizer_D_Monet = optim.Adam(D_Monet.parameters(), lr=1e-4, betas=(0.5, 0.5))

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()

scaler = GradScaler()

lambda_cycle = 5
num_epochs = 100
batch_size = 16

for epoch in range(num_epochs):
    epoch_loss_G_Photo2Monet = 0
    epoch_loss_G_Monet2Photo = 0
    epoch_loss_D_Photo = 0
    epoch_loss_D_Monet = 0
    
    for i, (real_photo, real_monet) in enumerate(zip(train_PhotoLoader, train_MonetLoader)):
        real_photo = real_photo.to(device)
        real_monet = real_monet.to(device)
        batch_size = real_photo.size(0)

        fake_monet = G_Photo2Monet(real_photo).detach()
        fake_photo = G_Monet2Photo(real_monet).detach()

        D_Photo_real = D_Photo(real_photo)
        D_Monet_real = D_Monet(real_monet)
        D_Photo_output_shape = D_Photo_real.shape
        D_Monet_output_shape = D_Monet_real.shape

        real_label_Photo = torch.ones(D_Photo_output_shape, dtype=torch.float, device=device)
        fake_label_Photo = torch.zeros(D_Photo_output_shape, dtype=torch.float, device=device)
        real_label_Monet = torch.ones(D_Monet_output_shape, dtype=torch.float, device=device)
        fake_label_Monet = torch.zeros(D_Monet_output_shape, dtype=torch.float, device=device)

        with autocast():
            D_Photo_real = D_Photo(real_photo)
            D_Photo_fake = D_Photo(fake_photo)
            loss_D_Photo_real = criterion_GAN(D_Photo_real, real_label_Photo)
            loss_D_Photo_fake = criterion_GAN(D_Photo_fake, fake_label_Photo)
            loss_D_Photo = (loss_D_Photo_real + loss_D_Photo_fake) * 0.5

            D_Monet_real = D_Monet(real_monet)
            D_Monet_fake = D_Monet(fake_monet)
            loss_D_Monet_real = criterion_GAN(D_Monet_real, real_label_Monet)
            loss_D_Monet_fake = criterion_GAN(D_Monet_fake, fake_label_Monet)
            loss_D_Monet = (loss_D_Monet_real + loss_D_Monet_fake) * 0.5

        optimizer_D_Photo.zero_grad()
        scaler.scale(loss_D_Photo).backward()
        scaler.step(optimizer_D_Photo)

        optimizer_D_Monet.zero_grad()
        scaler.scale(loss_D_Monet).backward()
        scaler.step(optimizer_D_Monet)

        epoch_loss_D_Photo += loss_D_Photo.item()
        epoch_loss_D_Monet += loss_D_Monet.item()

        with autocast():
            fake_monet = G_Photo2Monet(real_photo)
            fake_photo = G_Monet2Photo(real_monet)
            loss_GAN_Photo2Monet = criterion_GAN(D_Monet(fake_monet), real_label_Monet)
            loss_GAN_Monet2Photo = criterion_GAN(D_Photo(fake_photo), real_label_Photo)

            cycle_photo = G_Monet2Photo(fake_monet)
            cycle_monet = G_Photo2Monet(fake_photo)
            loss_cycle_photo = criterion_cycle(cycle_photo, real_photo)
            loss_cycle_monet = criterion_cycle(cycle_monet, real_monet)

            loss_G_Photo2Monet = loss_GAN_Photo2Monet + lambda_cycle * loss_cycle_photo
            loss_G_Monet2Photo = loss_GAN_Monet2Photo + lambda_cycle * loss_cycle_monet

        optimizer_G_Photo2Monet.zero_grad()
        scaler.scale(loss_G_Photo2Monet).backward()
        scaler.step(optimizer_G_Photo2Monet)

        optimizer_G_Monet2Photo.zero_grad()
        scaler.scale(loss_G_Monet2Photo).backward()
        scaler.step(optimizer_G_Monet2Photo)

        scaler.update()

        epoch_loss_G_Photo2Monet += loss_G_Photo2Monet.item()
        epoch_loss_G_Monet2Photo += loss_G_Monet2Photo.item()

    epoch_loss_G_Photo2Monet /= len(train_PhotoLoader)
    epoch_loss_G_Monet2Photo /= len(train_MonetLoader)
    epoch_loss_D_Photo /= len(train_PhotoLoader)
    epoch_loss_D_Monet /= len(train_MonetLoader)

    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"loss_G_Photo2Monet: {epoch_loss_G_Photo2Monet:.4f}, "
          f"loss_G_Monet2Photo: {epoch_loss_G_Monet2Photo:.4f}, "
          f"loss_D_Photo: {epoch_loss_D_Photo:.4f}, "
          f"loss_D_Monet: {epoch_loss_D_Monet:.4f}")

    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            G_Photo2Monet.eval()
            example_photo = next(iter(val_PhotoLoader)).to(device)
            fake_monet = G_Photo2Monet(example_photo)

            plt.figure(figsize=(12, 6))
            for i in range(4):
                plt.subplot(2, 4, i + 1)
                plt.imshow(((example_photo[i].cpu().numpy().transpose(1, 2, 0) + 1) / 2))
                plt.axis('off')
                plt.subplot(2, 4, i + 5)
                plt.imshow(((fake_monet[i].cpu().numpy().transpose(1, 2, 0) + 1) / 2))
                plt.axis('off')
            plt.show()

            G_Photo2Monet.train()

final_model_path = 'final_model.pth'
torch.save({
    'G_Photo2Monet': G_Photo2Monet.state_dict(),
    'G_Monet2Photo': G_Monet2Photo.state_dict(),
    'D_Photo': D_Photo.state_dict(),
    'D_Monet': D_Monet.state_dict(),
    'optimizer_G_Photo2Monet': optimizer_G_Photo2Monet.state_dict(),
    'optimizer_G_Monet2Photo': optimizer_G_Monet2Photo.state_dict(),
    'optimizer_D_Photo': optimizer_D_Photo.state_dict(),
    'optimizer_D_Monet': optimizer_D_Monet.state_dict(),
}, final_model_path)

In [ ]:
import shutil
from zipfile import ZipFile

device = "cuda" if torch.cuda.is_available() else "cpu"
model_path = '/kaggle/working/final_model.pth'
checkpoint = torch.load(model_path)


G_Photo2Monet = Generator(img_channels=3).to(device)
G_Photo2Monet.load_state_dict(checkpoint['G_Photo2Monet'])
G_Photo2Monet.eval()

test_loader = DataLoader(PhotoDataset, batch_size=1, shuffle=False)


os.makedirs('../tmp/images', exist_ok=True)

with torch.no_grad():
    for i, img in enumerate(test_loader):
        img = img.to(device)
        fake_monet = G_Photo2Monet(img)[0].cpu().permute(1, 2, 0).numpy() * 0.5 + 0.5
        im = Image.fromarray((fake_monet * 255).astype(np.uint8))
        im.save(f"../tmp/images/{i}.jpg")

output_zip_path = "/kaggle/working/images.zip"
with ZipFile(output_zip_path, 'w') as zipf:
    for root, _, files in os.walk('../tmp/images'):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), '../tmp/images'))

## Additional Resources

### Concepts of GANs
- [Generative Adversarial Networks(GANs): Complete Guide to GANs](https://www.analyticsvidhya.com/blog/2021/10/an-end-to-end-introduction-to-generative-adversarial-networksgans/)
- A CycleGAN operates through a structured process involving two core components: a generator and a discriminator. Both of these elements are built upon the foundation of conventional convolutional deep learning models. They utilize a series of layers designed to progressively reduce or augment the dimensions of an image. This intricate architecture allows the CycleGAN to effectively learn and transform images from one domain to another, leveraging the strengths of convolutional networks to manipulate image sizes through its generator and discriminator stages: [CycleGAN  |  TensorFlow Core](https://www.tensorflow.org/tutorials/generative/cyclegan)
- [Mohammad-Rahmdel/CycleGAN: Tensorflow 2 implementation of CycleGAN(Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks)](https://github.com/Mohammad-Rahmdel/CycleGAN)
- [LynnHo/CycleGAN-Tensorflow-2](https://github.com/LynnHo/CycleGAN-Tensorflow-2)
- CycleGAN | Tensorflow Core. TensorFlow. (n.d.). Retrieved October 7, 2022, from https://www.tensorflow.org/tutorials/generative/cyclegan
- Isola, P., Zhu, J.-Y., Zhou, T., & Efros, A. A. (2016). Image-to-image translation with conditional adversarial networks. Retrieved October 6, 2022, from Ανακτήθηκε από http://arxiv.org/abs/1611.07004
- Jang, A. (2020, August 29). Monet Cyclegan Tutorial. Kaggle. Retrieved October 7, 2022, from https://www.kaggle.com/code/amyjang/monet-cyclegan-tutorial/notebook
- nkmk. (n.d.). Convert BGR and RGB with python, opencv (cvtcolor). Convert BGR and RGB with Python, OpenCV (cvtColor). Retrieved October 7, 2022, from https://note.nkmk.me/en/python-opencv-bgr-rgb-cvtcolor/
- Stack Overflow. (2014, March 3). Python - calculate histogram of image. Stack Overflow. Retrieved October 6, 2022, from https://stackoverflow.com/questions/22159160/python-calculate-histogram-of-image
- Ulyanov, D., Vedaldi, A., & Lempitsky, V. (2016). Instance normalization: The missing ingredient for fast stylization. Retrieved October 6, 2022, from Ανακτήθηκε από http://arxiv.org/abs/1607.08022


### TensorFlow & Keras
- [TFRecord and tf.train.Example  |  TensorFlow Core](https://www.tensorflow.org/tutorials/load_data/tfrecord)

### PyTorch

- [Pytorch vs. TensorFlow: Which Framework to Choose? | by AnalytixLabs | Medium](https://medium.com/@byanalytixlabs/pytorch-vs-tensorflow-which-framework-to-choose-ed649d9e7a35)

### Python Coding
- [python - Limiting print output - Stack Overflow](https://stackoverflow.com/questions/29321969/limiting-print-output)
- [python - How to read (decode) .tfrecords file, see the images inside and do augmentation? - Stack Overflow](https://stackoverflow.com/questions/65007191/how-to-read-decode-tfrecords-file-see-the-images-inside-and-do-augmentation)
- [tensorflow/addons: Useful extra functionality for TensorFlow 2.x maintained by SIG-addons](https://github.com/tensorflow/addons#python-op-compatibility-matrix)
- [python - How to find which version of TensorFlow is installed in my system? - Stack Overflow](https://stackoverflow.com/questions/38549253/how-to-find-which-version-of-tensorflow-is-installed-in-my-system)
- [python - ImportError: No module named keras.optimizers - Stack Overflow](https://stackoverflow.com/questions/39081910/importerror-no-module-named-keras-optimizers)